In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense # creates densely connected layer object
from tensorflow.keras.layers import Flatten # takes 2D input and turns into 1D array

from tensorflow import keras
from tensorflow.keras import datasets, layers, models

from tensorflow.keras.layers import Conv2D # convolution layer
from tensorflow.keras.layers import MaxPooling2D # max pooling layer

from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation

from keras.preprocessing.image import ImageDataGenerator

In [3]:
train_dir_2 = 'Data/train_mel_and_noise'

test_dir = 'Data/mel_test_85_15'

In [4]:
train_data_gen_7_2 = ImageDataGenerator(rescale = 1./255, validation_split = .15)

train_generator_7_2 = train_data_gen_7_2.flow_from_directory(

                                                    train_dir_2, 
                                                    target_size = (200, 150),
                                                    batch_size = 32,
                                                    class_mode = 'categorical',
                                                    color_mode = 'rgb',
                                                    seed = 42,
                                                    subset = 'training',)

val_data_gen_7_2 = ImageDataGenerator(rescale=1./255)

validation_generator_7_2 = train_data_gen_7_2.flow_from_directory(
    
                                                    train_dir_2, 
                                                    target_size = (200, 150),
                                                    batch_size = 32,
                                                    class_mode = 'categorical',
                                                    color_mode = 'rgb',
                                                    seed = 42,
                                                    subset = 'validation',)

test_data_gen_7_2 = ImageDataGenerator(rescale=1./255)

test_generator_7_2 = test_data_gen_7_2.flow_from_directory(

                                                    test_dir, 
                                                    target_size = (200, 150),
                                                    batch_size = 32,
                                                    class_mode = 'categorical',
                                                    color_mode = 'rgb',
                                                    seed = 42,
                                                    shuffle = False)

In [9]:
custom_early_stopping = EarlyStopping(monitor='val_accuracy', patience=5) 
callbacks=[custom_early_stopping]

In [10]:
model7_2 = Sequential()

# define 3x3 filter window sizes. Create 32 filters.
model7_2.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(200, 150, 3), kernel_regularizer = l2(5e-4) ))

# max pool in 2x2 window
model7_2.add(MaxPooling2D(pool_size=(2, 2)))

# define 3x3 filter window sizes. Create 64 filters.
model7_2.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer = l2(5e-4)))
model7_2.add(MaxPooling2D((2, 2)))

model7_2.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer = l2(8e-4)))
model7_2.add(MaxPooling2D((2, 2)))
model7_2.add(Dropout(0.1))

model7_2.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer = l2(7e-3)))
model7_2.add(Dropout(0.25))

# transition to dense fully-connected part of network
model7_2.add(Flatten())
model7_2.add(Dense(128, activation='relu'))
model7_2.add(Dropout(0.4))

model7_2.add(BatchNormalization())
model7_2.add(Dense(64, activation='relu'))
model7_2.add(Dropout(0.7))

model7_2.add(Dense(10, activation='softmax'))

model7_2.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])

In [11]:
history_cnn_7_2 = model7_2.fit(train_generator_7_2, validation_data = validation_generator_7_2, 
                           epochs = 45, callbacks=[custom_early_stopping])